In [ ]:
import pandas as pd
import json
from pandas import json_normalize

In [ ]:
with open("csm_data/Registration_data_csm.json", "r") as f:
    regdata = json.load(f)

In [ ]:
columns = list(regdata[0].keys())
columns

In [ ]:
# Normalize the nested JSON
df = json_normalize(data)

In [ ]:
df.to_csv("csv/Registration_data_csm.csv", index=False)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
from datetime import datetime


def calculate_date_difference(df, date_column, given_date_str):
    # Convert the column to datetime
    df[date_column] = pd.to_datetime(df[date_column])

    # Convert the given date to datetime
    given_date = datetime.strptime(given_date_str, "%Y-%m-%d")

    # Calculate the difference in days
    df["Days_since_registration"] = (given_date - df[date_column]).dt.days

    return df

In [ ]:
# Calculate the difference in days from a given date
given_date_str = "2025-06-01"
result_df = calculate_date_difference(df, "RegistrationDate", given_date_str)

In [ ]:
result_df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df["ShowRef"].unique()

In [ ]:
df["BadgeType"].unique()

In [ ]:
df["Source"].unique()

In [ ]:
df["Status"].unique()

In [ ]:
shows_25 = ["BDAWL25", "CEEL25", "CCSEL25", "DCWL25", "DL25"]

In [ ]:
df["RegCode"].unique()

In [ ]:
df_reg_25 = df[df.ShowRef.isin(shows_25)]
len(df_reg_25)

In [ ]:
df_reg_25 = calculate_date_difference(df_reg_25, "RegistrationDate", given_date_str)
df_reg_25.head(2)

In [ ]:
def return_domain(x):
    return x.split("@")[-1]

In [ ]:
df_reg_25["Email"] = df_reg_25["Email"].apply(return_domain)

In [ ]:
df_reg_25.head(2)

In [ ]:
len(df_reg_25["BadgeId"].unique())

In [ ]:
df_reg_25.to_csv("csv/Registration_data_csm_25.csv", index=False)

In [ ]:
df_reg_25.columns

In [ ]:
columns = [
    "Email",
    "Company",
    "JobTitle",
    "Country",
    "BadgeType",
    "ShowRef",
    "BadgeId",
    "Source",
    "Days_since_registration",
]

In [ ]:
df_reg_25_valid_columns = df_reg_25[columns]

In [ ]:
df_reg_25_valid_columns.isnull().sum()

In [ ]:
df_reg_25_valid_columns.to_csv(
    "csv/Registration_data_csm_25_only_valid.csv", index=False
)

In [ ]:
with open("csm_data/Registration_demographicdata_csm.json", "r") as f:
    demodata = json.load(f)

In [ ]:
# Normalize the nested JSON
df2 = json_normalize(demodata)

In [ ]:
df2.to_csv("csv/Registration_demographicdata_csm.csv", index=False)

In [ ]:
df_demo_25 = df2[df2.showref.isin(shows_25)]
len(df_demo_25)

In [ ]:
df_demo_25.head()

In [ ]:
df_demo_25.to_csv("csv/Registration_demographicdata_csm_25.csv", index=False)

In [ ]:
len(df_demo_25["showref"].unique())

In [ ]:
len(df_demo_25["BadgeId"].unique())

In [ ]:
demo_badge_id_25 = list(df_demo_25["BadgeId"].unique())

In [ ]:
demo_badge_id_25[0]

In [ ]:
df_reg_25_wdemo_data = df_reg_25[df_reg_25["BadgeId"].isin(demo_badge_id_25)]
len(df_reg_25_wdemo_data)

In [ ]:
df_reg_25_valid_columns.isnull().sum()

In [ ]:
def concatenate_qa_registration_data(df_registration, list_badge_id_events):
    """
    Concatenate Registration Values of a Particpant"
    Args:
        df_registration : Pandas Dataframe with values of Participants
        list_badge_id_events : List with all BadgeId from which we have Geographic Data. Answer to the questionarie
    """

    def create_string_from_dict(data):
        keys_to_exclude = ["ShowRef", "BadgeId"]
        result = []

        for key, value in data.items():
            if key not in keys_to_exclude:
                if value is not None:
                    result.append(f"{key}: {value}")
                else:
                    result.append(f"{key}: no_data")

        return ", ".join(result)

    reg_data_list = []
    for badge_ID, i in zip(list_badge_id_events, range(len(list_badge_id_events))):
        df = df_registration[df_registration["BadgeId"] == badge_ID]
        df_d = json.loads(df.to_json(orient="records"))
        reg_data = {}
        for row in df_d:
            ID = "_".join([row.get("ShowRef"), row.get("BadgeId")])
            reg_data[ID] = create_string_from_dict(row)
        reg_data_list.append(reg_data)

    return reg_data_list

In [ ]:
reg_data_list = concatenate_qa_registration_data(
    df_reg_25_valid_columns, demo_badge_id_25
)

In [ ]:
len(reg_data_list), len(demo_badge_id_25)

In [ ]:
reg_data_list[1550]

In [ ]:
with open("output/registration_data.json", "w") as f:
    json.dump(reg_data_list, f, indent=4)

# check that we don't have same BadgeId in diferent showrefs

In [ ]:
result = df_demo_25.groupby(["BadgeId"])["showref"].nunique().reset_index()
result.columns = ["BadgeId", "Unique_showref_Count"]
result.head()

In [ ]:
result.info(verbose=True)

In [ ]:
result[result["Unique_showref_Count"] > 1]

# Choose which one Apply

In [ ]:
df_reg_25_wdemo_data["BadgeType"].unique()  # Choose which one Apply

In [ ]:
df_reg_25_wdemo_data.to_csv(
    "csv/Registration_data_with_demographicdata_csm_25.csv", index=False
)

In [ ]:
df_demo_25.isnull().sum()

# Concatenate Q&A Demografic Data

In [ ]:
list_no = [
    "I have read and accept the Event Terms and Conditions",
    "I have read and accept the Privacy Policy",
]
list_vip = [
    "What best describes your reason for attending: ‘I am attending because my company is...’",
    "Decision making power",
]

In [ ]:
for sentence in list_vip:
    if (
        "What best describes your reason for attending: ‘I am attending because my company is...’"[
            :20
        ]
        in sentence
    ):
        print("True")

In [ ]:
df_demo_25[df_demo_25["BadgeId"] == "J59MXE4"]

In [ ]:
def concatenate_qa_demografic_data(
    df_demografic, list_badge_id_events, list_no, list_vip
):
    """
    Concatenate Questions and Answers of a Particpant"
    """
    demo_data = []
    for badge_ID, i in zip(list_badge_id_events, range(len(list_badge_id_events))):
        df = df_demografic[df_demografic["BadgeId"] == badge_ID]
        df_d = json.loads(df.to_json(orient="records"))
        list_vipq, list_noq = [], []
        qa, qai = {}, {}
        OLD_ID = "_".join([df_d[0].get("showref"), df_d[0].get("BadgeId")])
        qa[OLD_ID] = ""
        qai[OLD_ID] = ""
        for row in df_d:
            ID = "_".join([row.get("showref"), row.get("BadgeId")])

            question = row.get("QuestionText").strip()
            answer = row.get("AnswerText").strip()
            if question in list_no:
                list_noq.append(f"Question: {question} not included")
                continue

            if ID == OLD_ID:
                qa_row = " : ".join([question, answer])
                found_vip = False
                for sentence in list_vip:
                    if question[:20].lower() in sentence.lower():
                        found_vip = True
                        break

                if found_vip:
                    list_vipq.append(f"Question: {question} is VIP")
                    qai[ID] = " , ".join([qai[ID], qa_row])
                else:
                    qa[ID] = " , ".join([qa[ID], qa_row])
            else:
                print("check_case")
                if question in list_vip:
                    qai[ID] = " , ".join([qai[ID], qa_row])
                else:
                    qa[ID] = " , ".join([qa[ID], qa_row])
                OLD_ID = ID
        # remove Initial padding
        qai[ID] = qai[ID][3:]
        qa[ID] = qa[ID][3:]
        # build final dictionary
        final = {}
        final[ID] = {}
        final[ID]["vip"] = qai[ID]
        final[ID]["normal"] = qa[ID]

        if len(list_vipq) != 2:
            print(f"Issue with records ID {ID} , row {i}, length dataframe: {len(df)}")
        demo_data.append(final)
        if i % 1000 == 0:
            print(f"Proccesed {i} records")
    return demo_data

In [ ]:
demo_data = concatenate_qa_demografic_data(
    df_demo_25, demo_badge_id_25, list_no, list_vip
)

In [ ]:
len(demo_data)

In [ ]:
# BDAWL25_NPL234P  106

In [ ]:
df_demo_25[df_demo_25["BadgeId"] == "CWQLTQ3"]

In [ ]:
demo_data[6765]

In [ ]:
demo_data[0].get("BDAWL25_J59MXE4").get("vip")

In [ ]:
demo_data[0].get("BDAWL25_J59MXE4").get("normal")

In [ ]:
with open("output/demographic_data.json", "w") as f:
    json.dump(demo_data, f, indent=4)

# Concatenate Scans to Demografic_data

In [ ]:
import pandas as pd
import json
from pandas import json_normalize

In [ ]:
with open("output/demographic_data.json", "r") as f:
    demo_data = json.load(f)

In [ ]:
demo_data[0].get("BDAWL25_J59MXE4").get("normal")

In [ ]:
merged_df = pd.read_csv(
    "csv/Registration_data_24_25_both_only_valid_badge_type_with_seminars.csv"
)

In [ ]:
merged_df.isnull().sum()

In [ ]:
len(merged_df)

In [ ]:
merged_df = merged_df.fillna("NA")
merged_df.head()

In [ ]:
list_badgesid_with_seminars = list(merged_df["BadgeId"].unique())

In [ ]:
for ele in demo_data:
    key = list(ele.keys())[0]
    badge_id = key.split("_")[-1]
    if badge_id in list_badgesid_with_seminars:

        temp = merged_df[merged_df["BadgeId"] == badge_id]
        temp_dict = json.loads(temp.to_json(orient="records"))
        ele[key]["Seminars"] = temp_dict[0].get("Concatenated Seminar Names")
        # print( temp_dict[0].get("Concatenated Seminar Names"))

In [ ]:
temp_dict

In [ ]:
ele[key]

In [ ]:
# for ele in demo_data:
#     key =list(ele.keys())[0]
#     keys = ele[key].keys()
#     if "Seminars" in keys:
#         print(ele[key]['Seminars'])

In [ ]:
with open("output/demographic_data_with_badge.json", "w") as f:
    json.dump(demo_data, f, indent=4)